In [36]:
from bs4 import BeautifulSoup
import lxml
import json
import rpy2.robjects as robj

In [37]:
allTheSoup = BeautifulSoup(open("/Users/brandonsherman/Documents/Sentimessenger/Sentimessenger/mymessages.html")
                          ,"lxml")

robj.r('''
getSenti <- function(string, sentiment){
	library(syuzhet)
	as.vector(get_nrc_sentiment(string)[1,])
}
'''
)

doSenti = robj.r("getSenti")


R object with classes: ('list',) mapped to:
<ListVector - Python:0x1198994c8 / R:0x1013593f8>
[Float..., Float..., Float..., ..., Float..., Float..., Float...]
  anger: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11989a0c8 / R:0x10813ad58>
[0.000000]
  anticipation: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11989a488 / R:0x1081307b8>
[0.000000]
  disgust: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11989a748 / R:0x108130938>
[0.000000]
  ...
  anger: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11989ad48 / R:0x108129b48>
[0.000000]
  anticipation: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11989d148 / R:0x108129cc8>
[1.000000

In [95]:
def parseOneMessageSoup(messageSoup, actualMessageSoup):
    message = {}
    message["User"] = messageSoup.find_all("span", class_ = "user")[0].get_text()
    message["Message"] = actualMessageSoup.get_text()
    message["Date"] = messageSoup.find_all("span", class_ = "meta")[0].get_text()
    calcedSentiments = doSenti(message["Message"])
    sentiments = {}
    sentiments["anger"] = tuple(calcedSentiments.rx2(1))[0]
    sentiments["anticipation"] = tuple(calcedSentiments.rx2(2))[0]
    sentiments["disgust"] = tuple(calcedSentiments.rx2(3))[0]
    sentiments["fear"] = tuple(calcedSentiments.rx2(4))[0]
    sentiments["joy"] = tuple(calcedSentiments.rx2(5))[0]
    sentiments["sadness"] = tuple(calcedSentiments.rx2(6))[0]
    sentiments["surprise"] = tuple(calcedSentiments.rx2(7))[0]
    sentiments["trust"] = tuple(calcedSentiments.rx2(8))[0]
    sentiments["negative"] = tuple(calcedSentiments.rx2(9))[0]
    sentiments["positive"] = tuple(calcedSentiments.rx2(10))[0]
    message["Sentiments"] = sentiments
    return(message)

def parseOneThreadSoup(threadSoup):
    allMessagesList = threadSoup.find_all("div", class_ = "message")
    actualMessages = threadSoup.find_all("p")
    if(len(allMessagesList) == len(actualMessages)):
        parsedThread = {}
        threadAsString = str(threadSoup)
        firstRightBracket = threadAsString.find(">",1)
        firstLeftBracket = threadAsString.find("<",1)
        members = threadAsString[firstRightBracket+1:firstLeftBracket]
        parsedThread["Members"] = members.split(", ")
        parsedMessages = []
        for i in range(0,len(allMessagesList)):
            parsedMessages.append(parseOneMessageSoup(allMessagesList[i],actualMessages[i]))
        parsedThread["Messages"] = parsedMessages
        #print(parsedThread)
        return(parsedThread)
    else:
        print("The number of p blocks didnt match number of messages")
        return(None)

def parseAllThreadsSoup(allThreads):
    allThreadsList = allThreads.find_all("div", class_ = "thread")
    parsedThreads = []
    for threadSoup in allThreadsList:
        parsedThreads.append(parseOneThreadSoup(threadSoup))
    return(parsedThreads)

In [96]:
parsedJSON = parseAllThreadsSoup(allTheSoup)

In [97]:
parsedJSON[0]

{'Members': ['Brandon Sherman',
  'Matthew Merrill',
  'Joey Laguna',
  'David Judilla',
  'Varun Ved'],
 'Messages': [{'Date': 'Wednesday, May 25, 2016 at 3:28pm PDT',
   'Message': 'J crew button ups for 29',
   'Sentiments': {'anger': 0.0,
    'anticipation': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'joy': 0.0,
    'negative': 0.0,
    'positive': 0.0,
    'sadness': 0.0,
    'surprise': 0.0,
    'trust': 1.0},
   'User': 'Joey Laguna'},
  {'Date': 'Wednesday, May 25, 2016 at 3:28pm PDT',
   'Message': 'Vbaby',
   'Sentiments': {'anger': 0.0,
    'anticipation': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'joy': 0.0,
    'negative': 0.0,
    'positive': 0.0,
    'sadness': 0.0,
    'surprise': 0.0,
    'trust': 0.0},
   'User': 'Joey Laguna'},
  {'Date': 'Wednesday, May 25, 2016 at 1:55pm PDT',
   'Message': "When you don't lock the door",
   'Sentiments': {'anger': 0.0,
    'anticipation': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'joy': 0.0,
    'negative': 0.0,
    'po